In [52]:
import json
import cv2
# import matplotlib.pyplot as plt
# from shapely.geometry import Polygon

with open("cropped_images/09_labelled/annotations/instances_default.json", "r") as read_file:
    instances = json.load(read_file)

full_img = cv2.imread("cropped_images/feb09_ps_actual.png")

In [73]:
i = 1
new_instances = {"licenses": [{"name": "", "id": 0, "url": ""}], "info": {"contributor": "", "date_created": "", "description": "", "url": "", "version": "", "year": ""}, "categories": [{"id": 1, "name": "Cracks", "supercategory": ""}, {"id": 2, "name": "Potholes", "supercategory": ""}, {"id": 3, "name": "Patches", "supercategory": ""}, {"id": 4, "name": "Sidewalk", "supercategory": ""}, {"id": 5, "name": "White Road Markings", "supercategory": ""}, {"id": 6, "name": "Double Yellow line", "supercategory": ""}, {"id": 7, "name": "Car", "supercategory": ""}, {"id": 8, "name": "Alligator Cracks", "supercategory": ""}], "images": [], "annotations": []}

for annotation in instances["annotations"]:
    origin = (int(instances["images"][annotation["image_id"]-1]["file_name"][:-4].split("_")[4]), int(instances["images"][annotation["image_id"]-1]["file_name"][:-4].split("_")[3]))
    defect_type = instances["categories"][annotation["category_id"]-1]["name"]
    defect_id = annotation["category_id"]
    segmentation = list(zip(*(iter(annotation["segmentation"][0]),) * 2))
    transformed_segmentation = [((point[0]*100+origin[0]*100)/100, (point[1]*100+origin[1]*100)/100) for point in segmentation]
    bounding_box = annotation["bbox"]
    transformed_bounding_box = bounding_box.copy()
    transformed_bounding_box[0] = (transformed_bounding_box[0]*100 + origin[0]*100)/100
    transformed_bounding_box[1] = (transformed_bounding_box[1]*100 + origin[1]*100)/100
    new_bbox_left = max(0, int(transformed_bounding_box[0]) - 100)
    new_bbox_right = min(14910, int(transformed_bounding_box[0]+transformed_bounding_box[2]) + 100)
    new_bbox_top = max(0, int(transformed_bounding_box[1]) - 100)
    new_bbox_bottom = min(1575, int(transformed_bounding_box[1]+transformed_bounding_box[3]) + 100)
    new_segmentation_points = [(round(point[0]*100-new_bbox_left*100)/100, round(point[1]*100-new_bbox_top*100)/100) for point in transformed_segmentation]
    new_segmentation = [coordinate for point in new_segmentation_points for coordinate in point]
    crop_img = full_img[new_bbox_top:new_bbox_bottom, new_bbox_left:new_bbox_right]
    filename = "cropped_" + str(i) + ".png"
    new_instances["images"].append({"id": i, "width": new_bbox_right-new_bbox_left, "height": new_bbox_bottom-new_bbox_top, "file_name": filename, "license": 0, "flickr_url": "", "coco_url": "", "date_captured": 0})
    new_instances["annotations"].append({"id": i, "image_id": i, "category_id": defect_id, "segmentation": [new_segmentation], "area": annotation["area"], "bbox": [round(transformed_bounding_box[0]*100-new_bbox_left*100)/100, round(transformed_bounding_box[1]*100-new_bbox_top*100)/100, annotation["bbox"][2], annotation["bbox"][3]], "iscrowd": 0, "attributes": {"occluded": False}})
    cv2.imwrite(filename, crop_img)
    i += 1

with open("new_instances.json", "w") as outfile:
    json.dump(new_instances, outfile)